# Char-based text generation with LSTM

In [ ]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Genertated from
'''
/content/drive/MyDrive/ML_Project4/input/test_chi/Steppenwolf_ch.txt
/content/drive/MyDrive/ML_Project4/input/new_prep_chi/三国演义_罗贯中_明_演义_ch.txt
/content/drive/MyDrive/ML_Project4/input/new_prep_chi/水浒传_施耐庵_明_小说_ch.txt
/content/drive/MyDrive/ML_Project4/input/new_prep_chi/One_Hundred_Years_of_Solitude_ch.txt
'''
with open('/content/drive/MyDrive/ML_Project4/input/new_prep_chi/One_Hundred_Years_of_Solitude_ch.txt','r') as text:
  train = text.read()
print(len(train))
train

319761


'第一章 多年 以后 奥雷连诺 上校 站 在 行刑队 面前 准 会 想起 父亲 带 他 去 参观 冰块 那个 遥远 下午 当时 马孔多 个 二十 户 人家 村庄 一座座 土房 盖 在 河岸 上 河水 清澈 沿着 遍布 石头 河床 流 去 河里 石头 光滑 洁白 活象 史前 巨蛋 这块 天地 还是 新 开辟 许多 东西 叫 不出 名字 不得不 用手 指指点点 每年 三月 衣衫 褴楼 吉卜赛人 要 在 村边 搭 起 帐篷 在 笛 鼓 喧嚣声 中向 马孔多 居民 介绍 科学家 最新 发明 他们 首先 带来 磁铁 一个 身躯 高大 吉卜赛人 自称 梅尔加 德斯 满脸 络腮胡子 手指 瘦得象 鸟 爪子 向 观众 出色 地 表演 他 所谓 马其顿 炼金术士 创造 世界 第八 奇迹 他 手里 拿 着 两大块 磁铁 从 一座 农舍 走到 另 一座 农舍 大家 惊异 地 看见 铁锅 铁盆 铁钳 铁炉 从 原地 倒下 木板 上 钉子 螺丝 嘎吱 嘎吱 地 拼命 想 挣脱 出来 甚至 那些 早就 丢失 东西 也 从 找 过 多次 地方 兀然 出现 乱七八糟 地 跟 在 梅尔加 德斯 魔 铁 后面 东西 也 有 生命 吉卜赛人 用 刺耳 声调 说 只消 唤起 它们 灵性 霍阿布 恩蒂亚 狂热 想象力 经常 超过 大自然 创造力 甚至 越过 奇迹 魔力 限度 他 认为 这种 暂时 无用 科学发明 可以 用来 开采 地下 金子 梅尔加 德斯 个 诚实 人 他 告诫 说 磁铁 干 这个 却 不行 可是 霍阿布 恩蒂亚 当时 还 不 相信 吉卜赛人 诚实 因此 用 自己 一匹 骡子 两只 山羊 换下 两块 磁铁 这些 家畜 他 妻子 打算 用来 振兴 破败 家业 她 试图 阻止 他 但是 枉费工夫 咱们 很快 会 有 足够 金子 用来 铺 家里 地 有余 啦 丈夫 回答 她 在 好 儿个 月 里 霍阿布 恩蒂亚 顽强 地 努力 履行 自己 诺言 他 带者 两块 磁铁 大声 地 不断 念 着 梅尔加 德斯 教 他 咒语 勘察 周围 整个 地区 一寸 寸 土地 甚至 河床 但 他 掘出 唯一 东西 十五世纪 一件 铠甲 它 各 部分 已锈得 连在一起 用手 一 敲皑甲 里面 发出 空洞 回声 仿佛 一只 塞满 石子 大 葫芦 三月 间 吉卜赛人 又 来 现在 他们 带来 一架 望远镜 一只 大小 似鼓

In [ ]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

In [ ]:
sequence, char_to_idx, idx_to_char = text_to_seq(train)

[' ', '他', '一', '在', '她', '不', '人', '上', '这', '个', '有', '到', '子', '里', '奥', '来', '时', '是', '雷', '们', '地', '阿', '诺', '连', '那', '自', '把', '着', '为', '说', '儿', '没', '得', '蒂', '过', '后', '出', '娜', '多', '家', '己', '兰', '大', '起', '去', '也', '生', '中', '可', '布', '然', '天', '发', '卡', '下', '象', '乌', '苏', '以', '第', '亚', '从', '房', '二', '种', '只', '霍', '间', '想', '样', '开', '看', '用', '要', '马', '么', '给', '经', '些', '斯', '恩', '现', '前', '于', '女', '特', '跟', '小', '情', '道', '最', '梅', '见', '还', '塔', '已', '会', '成', '能', '面', '手', '事', '心', '回', '年', '校', '因', '之', '达', '而', '使', '尔', '对', '死', '菲', '无', '头', '加', '但', '声', '玛', '次', '候', '信', '我', '进', '好', '几', '了', '知', '十', '列', '孔', '明', '当', '长', '气', '动', '身', '感', '向', '德', '又', '作', '你', '神', '两', '活', '走', '皮', '它', '打', '亲', '正', '点', '却', '比', '拉', '期', '行', '觉', '相', '法', '同', '老', '父', '望', '什', '让', '意', '至', '西', '战', '由', '的', '常', '带', '白', '分', '放', '所', '孩', '叫', '色', '门', '就', '全', '衣', '如', '听', '娘', '且', '花', '满', '住', '服', '母', '姑', '整', '并',

In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [ ]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)
from tqdm import tqdm
n_epochs = 1000 #12 mins for gpu to complete 10000 epochs
loss_avg = []

for epoch in tqdm(range(n_epochs)):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

  5%|▌         | 52/1000 [00:01<00:40, 23.44it/s]

Loss: 5.241649074554443
                                                                                                                                                                                                         


 10%|█         | 100/1000 [00:03<00:35, 25.32it/s]

Loss: 4.7299866771698
 他 上 一 他  他 他 他 在 在 他 他 他 他 他  他 他 把 不 一 一 他 他 一 上  他   他 他 他 他  他 他 一 他  自 一 他 他 一 他 他 他 他 然 在  他 他 他 不 他 他 他 一 地 他 他 他 他 他 他 他  他 他 在 他 他 他 一 他 他 他 他 上 他 他 这 他 一 他 在 他 他  他 一 一 他 他 他 在 他 他 他 他 他 他 他 


 15%|█▌        | 152/1000 [00:05<00:33, 25.39it/s]

...

Loss: 3.677835760116577
 一个 人 她 上 一个 人 不 在 一个 娜 不 她 里 姑娘 里 在 一个 人 一个 上 一个 地 一个 地 一个 人 他 蒂奥 上校 不 他 自己 她 在 不 他 上 在 一条 在 这个 人 一个 人 在 一个 人 一个 人 一个 娜 第二 一个 上校 一个 娜 上校 阿玛兰塔 一个 家里 那 她 一个 上 一个 娜 德斯 不 一个 人 他 上 把 一个 人 她 一个 上 一个 人 他 地 一


 95%|█████████▌| 952/1000 [00:31<00:01, 25.59it/s]

Loss: 3.6213782978057862
 上 不 一个 上 一个 上校 不 一个 人 一切 在 在 一个 地 在 一个 上校 一个 上 一个 上校 一个 人 一个 上 一个 上校 说 一个 上 一个 上校 不 这 这 一个 上校 他 一个 人 那 一个 上校 他 一个 上校 他 一个 人 在 一个 上校 一个 上校 他 用 他 在 一个 上 一个 人 他 她 在 他 他 也 阿卡 蒂奥 恩蒂亚 一个 人 去 一个 上 他 一个 上 一个 


100%|██████████| 1000/1000 [00:32<00:00, 30.46it/s]

Loss: 3.586367492675781
 在 她 在 他 他 这个 黛丝 一个 房子 他 这 他 他 他 他 在 他 在 他 一个 黛丝 他 这 他 他 一个 地 在 他 之后 他 这 这 他 在 他 这 一个 发现 她 一个 地 他 他 一个 一个 上 他 在 一个 地 在 一个 地 他 一个 地 他 他 在 在 一个 地 他 这 他 在 一个 地 他 他 奥雷连诺 第二 这 一个 房子 中 一个 头 他 在 他 他 在 这个 地 她 


In [ ]:
model.eval()
pred = evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=10000, 
    start_text='一个 星期 '
    )
print(pred)

一个 星期 蒂奥 一个 地 在 他 这 一个 地 一个 黛丝 他 他 这 这些 一个 地 不 在 这 一个 地 在 他 他 他 他 一个 地 这个 黛丝 他 他 一个 地 他 他 他 不 他 不是 他 他 一个 上 他 她 她 他 可是 他 他 在 在 一个 姑娘 里 这 他 发现 他 他 他 他 不 这 霍 阿玛兰塔 他 他 在 他 在 在 他 在 他 在 这 在 他 这个 房间 在 一个 地 这 一个 一个 地 这 一个 地 他 在 他 他 这 他 一个 地 在 他 在 他 这 他 他 他 他 阿玛兰塔 他 在 在 一个 地 地 在 她 在 这 一个 去 他 她 这 他 在 他 他 他 在 她 在 她 他 他 他 这 她 他 从 他 他 他 他 他 他 她 他 一个 人 他 他 一个 黛丝 他 他 他 一个 他 这些 地 他 在 他 这 一个 房间 中 他 他 他 他 不 她 在 他 在 他 在 这 他 这个 一个 黛丝 从 他 他 他 在 一个 姑娘 里 他 他 自己 不 他 一个 房间 他 这些 房子 他 他 在 他 他 他 一个 房子 她 在 这些 人 她 他 一个 地 一个 上校 一个 地 这 不 他 在 他 家里 他 这 他 她 在 一个 地 一个 黛丝 一个 一个 地 他 他 下 她 他 在 一个 上 他 在 一个 地 他 他 他 在 她 他 他 在 在 在 从 他 这 一个 地 他 这 一个 地 她 他 这 一个 这 一个 人 他 他 他 他 他 在 他 在 他 这 他 他 一个 房子 里 不是 这 他 他 在 他 他 一个 地 她 地 在 他 他 她 她 他 在 他 他 在 他 在 他 在 他 她 在 他 他 这 他 他 他 他 他 一个 地 这 他 她 他 他 在 一个 人 他 他 这 一个 地 他 在 自己 在 一个 人 这 一个 他 他 他 这 他 他 她 里 他 这 他 这个 上校 一个 地 在 这 一个 上校 她 一个 地 他 在 这个 人 一个 上 他 他 他 她 在 一个 人 不 他 他 他 他 在 不 一个 地 在 不是 他 一个 他 他 在 一个 地 一个 地 在 他 他 这  蒂亚 在 他 他 她 他 在 她 他 他 一个 地 她 他 他 地 在 他 把 他 一个 比 一个 地 在 一个 地 他 一个 地 他 一个 地 这 他

In [ ]:
saveFile = open('/content/drive/MyDrive/ML_Project4/sub/chi_bot3.txt', 'w')
saveFile.write(pred)

# saveFile.write(cleaned)
saveFile.close()

# English Bot Text

In [ ]:
'''
Texts used to generate the series:
('/content/drive/MyDrive/ML_Project4/input/test_eng_ben/Sonnets 31-60(Poetry).txt'  
/content/drive/MyDrive/ML_Project4/input/test_eng_ben/war_and_piece_en_maude_pre.txt #Not yet
/content/drive/MyDrive/ML_Project4/input/test_eng_ben/A Tale of Two Cities(Fiction).txt
'''

with open('/content/drive/MyDrive/ML_Project4/input/test_eng_ben/A Tale of Two Cities(Fiction).txt','r') as text:
  tmp= text.readlines()
train = []
for x in tmp:
    x = x.replace('\n','')
    if x == '' or x.isupper(): #Drop none and description of part of speech
      continue
    train.append(x.replace('\n',''))
train = ' '.join(train)
train

'chapter 1 book the first -- recalled to life chapter i the period it was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of light, it was the season of darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to heaven, we were all going direct the other way--in short, the period was so far like the present period, that some of its noisiest authorities insisted on its being received, for good or for evil, in the superlative degree of comparison only. there were a king with a large jaw and a queen with a plain face, on the throne of england; there were a king with a large jaw and a queen with a fair face, on the throne of france. in both countries it was clearer than crystal to the lords of the state preserves of loaves and fishes, that things in general were settle

In [ ]:
len(train)

852561

In [ ]:
sequence, char_to_idx, idx_to_char = text_to_seq(train)

[' ', 'e', 't', 'a', 'o', 'n', 'i', 'h', 's', 'r', 'd', 'l', 'u', 'm', 'c', 'w', 'f', ',', 'g', 'y', 'p', 'b', '.', 'v', '"', 'k', '-', "'", ';', '!', '?', 'j', 'q', 'x', ':', 'z', '’', '(', ')', '`', '1', '3', '2', '7', '4', '—', '_', '5', '0', '6', '9', '“', '”', '8', '#', 'é', ']', '>', '&', '/', '–', '…', 'ç']


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)
from tqdm import tqdm
loss_avg = []
n_epochs = 1000 #12 mins for gpu to complete 10000 epochs

for epoch in tqdm(range(n_epochs)):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

  5%|▌         | 53/1000 [00:00<00:18, 50.28it/s]

Loss: 2.5846115159988403
 and here foun with and the the he and be corres and lith wher and and the the the the the woun he the and the and the and in and wang the the houn the the corre and fat the the and the wore and and an


 10%|█         | 100/1000 [00:01<00:17, 50.10it/s]

Loss: 2.057916166782379
 of the see the and ming the the and the preat it the had the for the of the and made and and and the stressent he the care her should and and had the the ston in the so storn the and and the and the f


 16%|█▌        | 155/1000 [00:02<00:15, 53.36it/s]

...

Loss: 1.4399040818214417
 the respect of the doctor manette to the tribunal pross that he had been in the man the prisoner that the prisoner of the strong and mened the men of the man and been a house and the word the man the 


 95%|█████████▍| 949/1000 [00:15<00:00, 65.32it/s]

Loss: 1.4448871064186095
 sire the end the soldiers of the court of the concertain of the court of the street and so could not the day and so can it was the court of the house in the signed and should be more that he was a con


100%|██████████| 1000/1000 [00:16<00:00, 60.14it/s]

Loss: 1.4328161597251892
 the face of the straight of the repeated to the prison and read of the republic the same excepting the and the streets of the days and said the country of the court to strick of the man when the man o


In [ ]:
model.eval()
pred = evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=10000, 
    start_text='the strick of the man  ' 
    )
print(pred)

the strick of the man  a man and the republic the court of the sunderty, and the doctor manette and the doctor manette said the bring the remaining him and respect and should be considered him the stranger and the man with his spoken, and she is the is the doctor manette said the street the man and straight of the more than the two with his fine to the stranger of his face to say the court of the marquis of the surprise of the counter and to be so father the considered the court of his changed and stranger and the street, and the man and the dissible and the not the country and streets of the morning the struck to the topition that i have been the streets that he had been side and so many to side the marquis hand the counter of the morning the man the conterrished the more for the street, and he could the streets to the change of the street of the streets of the sounded to his house, and have been a considered to him, and the men that he was the man strike the morning many and the chan

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/ML_Project4/input/text_gen_model/eng_textcnn.pth')

In [ ]:
# torch.load('/content/drive/MyDrive/ML_Project4/input/text_gen_model/eng_textcnn.pth')

In [ ]:
saveFile = open('/content/drive/MyDrive/ML_Project4/sub/eng_bot2.txt', 'w')
saveFile.write(pred)
saveFile.close()

# Russian bot text

In [ ]:
import re
'''
Texts used:
/content/drive/MyDrive/ML_Project4/input/test_ru_ben/122.1.1.1.17.txt
/content/drive/MyDrive/ML_Project4/input/test_ru_ben/122.1.1.3.4.1.txt
/content/drive/MyDrive/ML_Project4/input/test_ru_ben/4.1.2.2.3.6.txt
'''
with open('/content/drive/MyDrive/ML_Project4/input/test_ru_ben/122.1.1.1.17.txt','r') as text:
  tmp = text.readlines()
train = []
for sen in tmp:
    sen = sen.replace('\n','')
    if sen == '' or sen.isalpha() or sen == 'MUSEUM ANAТОМICUM2':
      continue
    train.append(sen)

train = ' '.join(train)
train

'"Oro supplex ei acclinis, Cor conlritum quasi cinis: Cere curam mei finis!" Requiem, Confutatis. Познав все нищенство земных великолепий, Мы вместе тешились чудовищной игрой... Мы откровение искали в тихом склепе, Нам проповедовал скелетов важный строй... Мудрей что может быть?.. Что может быть нелепей?.. Мой взор прикован был старинною гравюрой, и был семнадцатый на ней означен век... Готических окон чуть брезжил сумрак хмурый в тот час, когда планет медлительный разбег, и первый, бледный луч, блуждая за решеткой, на каменной стене, черневшей, словно снег на людной улице, отбросил контур четкий, зловеще удлинив рогов оленьих тень, – и все двоилось там, меж окон посередке, везде, склонив рога. являлся мне олень. Все уносило там мечту к средневековью, вкруг знаки странные читал пугливый день, и человеческой, горячей пахло кровью... Странным склепом мне казался тихий зал, и надпись, что была расписана с любовью, мой изумленный взор с усильем разобрал: "Museum anatomicum, instrumentale".

In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 10
sequence, char_to_idx, idx_to_char = text_to_seq(train)

[' ', 'о', 'е', 'а', 'н', 'с', 'л', 'и', 'т', 'в', 'р', 'м', ',', 'к', 'д', '.', 'ы', 'я', 'з', 'у', 'й', 'ь', 'г', 'п', 'б', 'ч', 'ж', 'х', 'ш', 'i', '!', 's', 'e', 'u', '"', 'm', 'a', 'n', 'r', 'ю', 'o', 't', 'щ', 'ц', 'c', 'Н', 'l', ':', 'К', '–', 'В', 'М', 'И', 'О', 'p', 'C', 'q', '?', 'Ч', 'Г', 'С', 'M', '(', ')', 'э', 'Т', 'b', 'f', 'П', '-', 'З', 'ф', 'А', 'N', ';', 'O', 'x', 'R', 'Ф', 'Я', 'I', 'g', 'Е', 'Х']


In [ ]:
len(train)

4484

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

loss_avg = []
n_epochs = 1000
for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.784458050727844
 раченне тревиг в беня тор мень, и и мерто в тет не в не рам ковей ней стрет в скорал пертал недой в скулене менне в от в строго не на сторь вено в стело стрет на мень, в тет на стол скорит мень мен та
Loss: 1.7250290131568908
 давно страшная и в средтал тогда предсная пылала тредь от меня, что стал скелета плечеста меня, и был он устрашный страшный страшный страшный страшный страшный страшный страшный страшный страшная и в 
Loss: 0.8213310241699219
 была расписана с морокных взора, и мне было тажес рых редов от меня два мне ветвей, злорадно извиваясь, надо мной глухой качнулся свод, надо мной улице, отбросил когда Господен зов достешились ту проч
Loss: 0.31347637295722963
 он вещая света, Архангел надо мной глухой качнулся свод, надо мной нежданно обрываясь. И был он весь обрит, и кровь с него текла, по камням медленно струясь и застывая, как стынет в сумерках горячая с
Loss: 0.1425796253979206
 надпись, что была расписана с любовью, мой изумленный взор с усиль

In [ ]:
model.eval()
pred = evaluate(
    model, 
    char_to_idx,  
    idx_to_char, 
    temp=0.3, 
    prediction_len=10000, 
    start_text='Не знаю, был то бред, иль'
    )
print(pred)

Не знаю, был то бред, иль словно владыкой света. Гласила первая: "Nos summus – umbra!.." Там, за ней тянулася еще, прося ответа: – "In nobis nosce te!.." За ней еще очам явилась страшная и вечная загадка: "Mors – rerum ultima est linea!.." Но сам, от страха трепеща, я все ж прочел украдкой: "Mors sceptra omnia ligonibus aequat!.." Как надпись ту прочесть в тот миг мне было сладко! "Nascentes morimur!" прочел на третьей взгляд... Ее держал скелет оскаленный ребенка; когда же взор отвел в смущенье я назад, мне вдруг почудилось, что он хохочет звонко. Но все сильней заря пылала, и в окне плясал пылинок рой, решетки контур тонкий яснел на каменной, заплесневшей стене; я снова бросил взор на мертвеца немого, и мысль безумная тогда предстала мне (Хоть выразить ту мысль теперь бессильно слово!..) В его чертах я вмиг узнал свои черты и весь похолодел от вихря ледяного... – "Чего дрожишь? Ведь мы – одно, и я, и ты!" Казалось, говорил мне труп недвижным взглядом и звал меня, презрев пугливые меч

In [ ]:
saveFile = open('/content/drive/MyDrive/ML_Project4/sub/ru_bot.txt', 'w')
saveFile.write(pred)
saveFile.close()